<a href="https://colab.research.google.com/github/TonySaad-CENA/Bootstrap_Website/blob/master/GPT_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/openai/gpt-2.git

Cloning into 'gpt-2'...
remote: Enumerating objects: 233, done.
remote: Total 233 (delta 0), reused 0 (delta 0), pack-reused 233
Receiving objects: 100% (233/233), 4.38 MiB | 21.45 MiB/s, done.
Resolving deltas: 100% (124/124), done.


In [2]:
import os
os.chdir("gpt-2")

In [3]:
!pip install tensorflow==1.14.0

     |████████████████████████████████| 109.2MB 98kB/s 
     |████████████████████████████████| 491kB 51.7MB/s 
     |████████████████████████████████| 3.2MB 48.8MB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.0
    Uninstalling tensorboard-2.4.0:
      Successfully uninstalled tensorboard-2.4.0
  Found existing installation: tensorflow 2.4.0
    Uninstalling tensorflow-2.4.0:
      Successfully uninstalled tensorflow-2.4.0


In [4]:
!pip3 install -r requirements.txt

     |████████████████████████████████| 81kB 7.5MB/s 
     |████████████████████████████████| 604kB 18.9MB/s 
     |████████████████████████████████| 61kB 9.0MB/s 
     |████████████████████████████████| 51kB 8.4MB/s 
     |████████████████████████████████| 61kB 10.5MB/s 
  Created wheel for fire: filename=fire-0.3.1-py2.py3-none-any.whl size=111006 sha256=003ab1a8bca59708b9a4e28e9607ca7613aa5a40ce0761000cc067e0f3c0bf75
  Stored in directory: /root/.cache/pip/wheels/c1/61/df/768b03527bf006b546dce284eb4249b185669e65afc5fbb2ac
  Created wheel for regex: filename=regex-2017.4.5-cp36-cp36m-linux_x86_64.whl size=533203 sha256=61fd4944414a1db28806446fc72ed08edff987acc718d68bdfeeb22c88b74550
  Stored in directory: /root/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built fire regex
ERROR: spacy 2.2.4 has requirement tqdm<5.0.0,>=4.38.0, but you'll have tqdm 4.31.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, bu

In [5]:
!python3 download_model.py 345M

Fetching checkpoint: 1.00kit [00:00, 1.08Mit/s]                                                     
Fetching encoder.json: 1.04Mit [00:00, 3.90Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 879kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:41, 34.4Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 10.3Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 4.10Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 2.62Mit/s]                                                       


In [6]:
!export PYTHONIOENCODING=UTF-8

In [7]:
os.chdir('src')

In [8]:
import json
import os
import numpy as np
import tensorflow as tf
import model, sample, encoder

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [9]:
def interact_model(
    model_name,
    seed,
    nsamples,
    batch_size,
    length,
    temperature,
    top_k,
    models_dir
):
    models_dir = os.path.expanduser(os.path.expandvars(models_dir))
    if batch_size is None:
        batch_size = 1
    assert nsamples % batch_size == 0

    enc = encoder.get_encoder(model_name, models_dir)
    hparams = model.default_hparams()
    with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if length is None:
        length = hparams.n_ctx // 2
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.Session(graph=tf.Graph()) as sess:
        context = tf.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        output = sample.sample_sequence(
            hparams=hparams, length=length,
            context=context,
            batch_size=batch_size,
            temperature=temperature, top_k=top_k
        )

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
        saver.restore(sess, ckpt)

        while True:
            raw_text = input("Model prompt >>> ")
            while not raw_text:
                print('Prompt should not be empty!')
                raw_text = input("Model prompt >>> ")
            context_tokens = enc.encode(raw_text)
            generated = 0
            for _ in range(nsamples // batch_size):
                out = sess.run(output, feed_dict={
                    context: [context_tokens for _ in range(batch_size)]
                })[:, len(context_tokens):]
                for i in range(batch_size):
                    generated += 1
                    text = enc.decode(out[i])
                    print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
                    print(text)
            print("=" * 80)

In [ ]:
interact_model(
    '345M',
    None,
    1,
    1,
    300,
    1,
    0,
    '/content/gpt-2/models'
)

INFO:tensorflow:Restoring parameters from /content/gpt-2/models/345M/model.ckpt
Model prompt >>> I went to a lounge to celebrate my birthday and
======================================== SAMPLE 1 ========================================
 there was a Cyberfood ticket for Joanna. the owner of her cafe was slightly embarrassed that I was standing there. when she heard that's what had happened she smiled a bit and said "is that how it is in Berlin :)". nope though she did not have a Derby shirt on her anyway. only a cheap 60€ fan.

Foto I arrived lol everyone had a picture of me there...

Stein # 2: thematic Cheese and Alebbalbar in borrowed style in parliament city the added Nasia Ms. Mahorgnaza thanks thanks so much , rever en mesgång.

Lady Queer Belle : Kieri for the Finnhardt Errock!

Reader siberian: JA X Avini Apuzzi;lion tears every punch!

Daniel Spa Corner : cross plank blow. love this dress spun hasopas curves! Kley.

rose foreign author: fant(u graphic)ze celeb detectives,. Dins